In [1]:
import numpy as np
import pandas as pd
from math import radians, cos, sin, asin, sqrt
from datetime import datetime

In [2]:
# Fetching the data
# I have added a column on my own- population_density per sq km. (I have done my research and mapped the population
#density per km for every city. Explained later under major logic)

df1= pd.read_csv("cities.csv")
df2= pd.read_csv("climate.csv")

In [3]:
# looking at the columns and the head values
df1.head()

,city,lat,lng,country,iso2,admin,capital,population,population_proper,population_density per sq km
0,Toronto,43.666667,-79.416667,Canada,CA,Ontario,admin,5213000,3934421,4149.5
1,Montréal,45.500000,-73.583333,Canada,CA,Québec,NaN,3678000,2356556,898.1
2,Vancouver,49.250000,-123.133333,Canada,CA,British Columbia,NaN,2313328,603502,5400.0
3,Ottawa,45.416667,-75.700000,Canada,CA,Ontario,primary,1145000,812129,317.0
4,Calgary,51.083333,-114.083333,Canada,CA,Alberta,NaN,1110000,915322,1329.0


In [4]:
df2.head()

,lng,lat,STATION_NAME,CLIMATE_IDENTIFIER,ID,LOCAL_DATE,PROVINCE_CODE,LOCAL_YEAR,LOCAL_MONTH,LOCAL_DAY,...,SPEED_MAX_GUST,SPEED_MAX_GUST_FLAG,COOLING_DEGREE_DAYS,COOLING_DEGREE_DAYS_FLAG,HEATING_DEGREE_DAYS,HEATING_DEGREE_DAYS_FLAG,MIN_REL_HUMIDITY,MIN_REL_HUMIDITY_FLAG,MAX_REL_HUMIDITY,MAX_REL_HUMIDITY_FLAG
0,-112.966667,54.283333,ABEE AGDM,3010010,3010010.2020.1.1,2021-01-01 0:00,AB,2021,1,1,...,NaN,NaN,0.0,NaN,27.2,NaN,86.0,NaN,97.0,NaN
1,-112.966667,54.283333,ABEE AGDM,3010010,3010010.2020.1.2,2021-01-02 0:00,AB,2021,1,2,...,53.0,NaN,0.0,NaN,25.9,NaN,47.0,NaN,99.0,NaN
2,-112.966667,54.283333,ABEE AGDM,3010010,3010010.2020.1.3,2021-01-03 0:00,AB,2021,1,3,...,37.0,NaN,0.0,NaN,30.0,NaN,77.0,NaN,93.0,NaN
3,-112.966667,54.283333,ABEE AGDM,3010010,3010010.2020.1.4,2021-01-04 0:00,AB,2021,1,4,...,65.0,NaN,0.0,NaN,23.1,NaN,64.0,NaN,92.0,NaN
4,-112.966667,54.283333,ABEE AGDM,3010010,3010010.2020.1.5,2021-01-05 0:00,AB,2021,1,5,...,NaN,NaN,0.0,NaN,28.6,NaN,57.0,NaN,98.0,NaN


In [5]:
# checking the null values
df1.isna().sum()

city                              0
lat                               0
lng                               0
country                           0
iso2                              0
admin                             0
capital                         233
population                        0
population_proper                 0
population_density per sq km      0
dtype: int64

In [6]:
df2.isna().sum()

lng                             0
lat                             0
STATION_NAME                    0
CLIMATE_IDENTIFIER              0
ID                              0
LOCAL_DATE                      0
PROVINCE_CODE                   0
LOCAL_YEAR                      0
LOCAL_MONTH                     0
LOCAL_DAY                       0
MEAN_TEMPERATURE             2057
MEAN_TEMPERATURE_FLAG       39870
MIN_TEMPERATURE              1940
MIN_TEMPERATURE_FLAG        39987
MAX_TEMPERATURE              1975
MAX_TEMPERATURE_FLAG        39952
TOTAL_PRECIPITATION         11072
TOTAL_PRECIPITATION_FLAG    31905
TOTAL_RAIN                  33618
TOTAL_RAIN_FLAG             40522
TOTAL_SNOW                  33579
TOTAL_SNOW_FLAG             36864
SNOW_ON_GROUND              19648
SNOW_ON_GROUND_FLAG         41116
DIRECTION_MAX_GUST          23110
DIRECTION_MAX_GUST_FLAG     35304
SPEED_MAX_GUST              22928
SPEED_MAX_GUST_FLAG         35485
COOLING_DEGREE_DAYS          2057
COOLING_DEGREE

MAJOR LOGIC- :

Values Longitudes and Latitudes given in cities.csv and climate.csv are different. This logic map the two dataframes together.

I am assuming that all the cities are spherical (circular on map) in shape and urban population resides inside the city and rural population resides on the outskirts. So we can mark a specific radius for all the cities till where the urban population resides. That radius will be my "threshold". It is different for every city as every city has different area. 

Now, once the threshold/ radius has been decided, we can see which all Stations are coming within that radius range. This will mean that those stations were recording the mean temperature for urban population and for the area within the radius.

Foe example -: Sudbury had 47.1 population per sq. km. This means that if 47.1 people lies is 1 sq km. Urban Population of 80507 (Mean_Proper) will take (1/ 47.1)*80507 . Assuming Sudbury is circular, then radius will be sqrt(((1/47.1)*80507) / 3.14) as area is (3.14 * r^2).


Then distance between station coordinates and city coordinates can be found out. I have used the following calculations-:


Find the value of the latitude in radians:
Value of Latitude in Radians, lat = Latitude / (180/pi) OR
Value of Latitude in Radians, lat = Latitude / 57.29577951
Find the value of longitude in radians:
Value of Longitude in Radians, long = Longitude / (180/pi) OR
Value of Longitude in Radians, long = Longitude / 57.29577951

Get the co-ordinates of point A in terms of latitude and longitude. Use the above conversion method to convert the values of latitude and longitude in radians. I will call it as lat1 and long1. Do the same for the co-ordinates of Point B and get lat2 and long2.
Now, to get the distance between point A and point B use the following form

Distance, d = 3963.0 * arccos[(sin(lat1) * sin(lat2)) + cos(lat1) * cos(lat2) * cos(long2 – long1)]

The obtained distance, d, is in miles. If you want your value to be in units of kilometers, multiple d by 1.609344.
d in kilometers = 1.609344 * d in miles [1]



Reference-:
[1]: https://www.geeksforgeeks.org/program-distance-two-points-earth/

In [7]:
def make_threshold(df1):
    # defining the threshold value
    df1["radius"]= np.sqrt(((1/df1["population_density per sq km"])*df1["population_proper"])/3.14)
    for i in range(0,len(df1),1):
        # if the radius is more than 100, I am taking it as 100 so that there is lesser overlapping between 2 cities when mapping stations with cities
        if(df1.iloc[i,-1] > 100):
            df1.iloc[i,-1] = 100
        
    return df1

make_threshold(df1)

,city,lat,lng,country,iso2,admin,capital,population,population_proper,population_density per sq km,radius
0,Toronto,43.666667,-79.416667,Canada,CA,Ontario,admin,5213000,3934421,4149.50,17.377116
1,Montréal,45.500000,-73.583333,Canada,CA,Québec,NaN,3678000,2356556,898.10,28.907578
2,Vancouver,49.250000,-123.133333,Canada,CA,British Columbia,NaN,2313328,603502,5400.00,5.965923
3,Ottawa,45.416667,-75.700000,Canada,CA,Ontario,primary,1145000,812129,317.00,28.563936
4,Calgary,51.083333,-114.083333,Canada,CA,Alberta,NaN,1110000,915322,1329.00,14.810156
5,Edmonton,53.550000,-113.500000,Canada,CA,Alberta,admin,1058000,712391,123.00,42.947890
6,Hamilton,43.256101,-79.857484,Canada,CA,Ontario,NaN,721053,519949,1371.00,10.989978
7,Winnipeg,49.883333,-97.166667,Canada,CA,Manitoba,admin,632063,575313,1430.00,11.319292
8,Québec,46.800000,-71.250000,Canada,CA,Québec,admin,624177,528595,5.80,100.000000
9,Oshawa,43.900000,-78.866667,Canada,CA,Ontario,NaN,450963,247989,420.60,13.703034


In [8]:
# mapping fucntion is used to map cities to stations - which stations come within the threshold defined for each city
# if the City in df2 value after the funtion remains 0, that mean I am assuming that particular weather station is getting temperature for rural cities.

In [9]:
def mapping(df1, df2):
    # I noticed there were some null values in the column MEAN_TEMPERATURE, so simply back filling it.
    df2.MEAN_TEMPERATURE.fillna(method = "bfill", inplace = True)
    
    df2["CITY"]= 0   # making column CITY for further use
    df2["E_distance"]=-1   # making column E_distance for further use
    
    for i in range(0, len(df1), 1):
        for j in range (0, len(df2),1):
            
            # checking the distance between station id and cities
            x_lat= df1.loc[i,"lat"] / 57.29577951
            y_lat= df2.loc[j,"lat"] / 57.29577951
            x_lng= df1.loc[i,"lng"] / 57.29577951
            y_lng= df2.loc[j,"lng"] / 57.29577951
            
            dlon = y_lng - x_lng
            dlat = y_lat - x_lat
            
            
            a = sin(dlat / 2)**2 + cos(x_lat) * cos(y_lat) * sin(dlon / 2)**2
            
            c = 2 * asin(sqrt(a))
            
            # Radius of earth in kilometers. 
            r = 6371
      
            # calculate the result
            distance= c * r
            
            if(distance < 0):
                distance= distance*-1
            
            # if the starion comes within the range of the threshold of the city, I am mapping the city with that station    
            if (distance < df1.loc[i,"radius"]):
                if(df2.loc[j,"E_distance"] != -1 and distance < df2.loc[j, "E_distance"]):
                    df2.loc[j,"CITY"] = df1.loc[i, "city"]
                    df2.loc[j,"E_distance"]= distance                
                
                elif(df2.loc[j,"E_distance"] == -1):
                    df2.loc[j,"CITY"] = df1.loc[i, "city"]
                    df2.loc[j,"E_distance"]= distance
                    
    return df1, df2

In [14]:
# this is the methid asked in the question
def display_result(df1,df2):
    df1= make_threshold(df1)
    
    #This function line 8 (mapping function) is taking 10 -20 mins to run. Since I want to save your time I am fetching the result of this funciton 
    # from map.csv. I saved the result earlier using pd.to_csv function. If you want to run it. Please uncomment it and run but then comment 12 and 13 
    
    #df1,df2= mapping(df1,df2)
    
    
    # if you want to run the line 8 (mapping function)- pls comment lines 12 and 13
    df2= pd.read_csv("map.csv")
    df2.drop("Unnamed: 0", axis=1, inplace=True)
     
    date_time_str = input("Enter the date in dd/mm/yy")
    date = datetime. strptime(date_time_str, '%d/%m/%Y')# getting the date
    day= date.day
    month= date.month
    year= date.year
    
    # pandas to display the result
    df_result= df2[(df2.LOCAL_YEAR== year) & (df2.LOCAL_MONTH==month) & (df2.LOCAL_DAY==day) & (df2.CITY!=0)]

    if(len(df_result)==0):
        print("No Records for this date")
        return 0

    mean_value = df_result.MEAN_TEMPERATURE.mean()
    median_value= df_result.MEAN_TEMPERATURE.median()



    print("Mean_Value is ", mean_value)
    print("Median Value is ", median_value)

In [20]:
# this is the main function to run.
display_result(df1,df2)

Enter the date in dd/mm/yy01/01/2021
Mean_Value is  -5.174553571428571
Median Value is  -3.3


In [17]:
# cleaning the dataframe
df1.drop(["country", "iso2", "admin"], axis=1, inplace =True)
    
    
    
     
# cleaning the dataframe
df2.drop(['CLIMATE_IDENTIFIER', 'ID', 'PROVINCE_CODE', 'MEAN_TEMPERATURE_FLAG', 'MIN_TEMPERATURE',
'MIN_TEMPERATURE_FLAG', 'MAX_TEMPERATURE', 'MAX_TEMPERATURE_FLAG',
'TOTAL_PRECIPITATION', 'TOTAL_PRECIPITATION_FLAG', 'TOTAL_RAIN',
'TOTAL_RAIN_FLAG', 'TOTAL_SNOW', 'TOTAL_SNOW_FLAG', 'SNOW_ON_GROUND',
'SNOW_ON_GROUND_FLAG', 'DIRECTION_MAX_GUST', 'DIRECTION_MAX_GUST_FLAG',
'SPEED_MAX_GUST', 'SPEED_MAX_GUST_FLAG', 'COOLING_DEGREE_DAYS',
'COOLING_DEGREE_DAYS_FLAG', 'HEATING_DEGREE_DAYS',
'HEATING_DEGREE_DAYS_FLAG', 'MIN_REL_HUMIDITY', 'MIN_REL_HUMIDITY_FLAG',
'MAX_REL_HUMIDITY', 'MAX_REL_HUMIDITY_FLAG'],axis=1, inplace= True )

In [18]:
# final Dataframe
df1.head()

,city,lat,lng,capital,population,population_proper,population_density per sq km,radius
0,Toronto,43.666667,-79.416667,admin,5213000,3934421,4149.5,17.377116
1,Montréal,45.500000,-73.583333,NaN,3678000,2356556,898.1,28.907578
2,Vancouver,49.250000,-123.133333,NaN,2313328,603502,5400.0,5.965923
3,Ottawa,45.416667,-75.700000,primary,1145000,812129,317.0,28.563936
4,Calgary,51.083333,-114.083333,NaN,1110000,915322,1329.0,14.810156


In [19]:
# Final dataframe
df2.head()

,lng,lat,STATION_NAME,LOCAL_DATE,LOCAL_YEAR,LOCAL_MONTH,LOCAL_DAY,MEAN_TEMPERATURE,CITY,E_distance
0,-112.966667,54.283333,ABEE AGDM,2021-01-01 0:00,2021,1,1,-9.2,0,-1.0
1,-112.966667,54.283333,ABEE AGDM,2021-01-02 0:00,2021,1,2,-7.9,0,-1.0
2,-112.966667,54.283333,ABEE AGDM,2021-01-03 0:00,2021,1,3,-12.0,0,-1.0
3,-112.966667,54.283333,ABEE AGDM,2021-01-04 0:00,2021,1,4,-5.1,0,-1.0
4,-112.966667,54.283333,ABEE AGDM,2021-01-05 0:00,2021,1,5,-10.6,0,-1.0
